In [1]:

import os, os.path
import numpy as np
import pandas as pd
import model_attributes as ma
from attribute_table import AttributeTable
import model_afolu as mafl
import model_ippu as mi
import model_circular_economy as mc
import model_energy as me
import model_electricity as ml
import model_socioeconomic as se
from model_socioeconomic import Socioeconomic
import re
import setup_analysis as sa
import sisepuede as ssp
import support_classes as sc
import support_functions as sf
import importlib
import time
import warnings
import matplotlib.pyplot as plt
import sql_utilities as sq
from typing import *
import sqlalchemy
import sql_utilities as sqlutil
import time
importlib.reload(ma)
importlib.reload(sa)
importlib.reload(sf)
importlib.reload(mafl)
importlib.reload(mc)
importlib.reload(mi)
importlib.reload(me)
importlib.reload(se)
importlib.reload(ml)

warnings.filterwarnings("ignore")

import logging
def _setup_logger(namespace: str, fn_out: Union[str, None] = None) -> None:
    global logger
    
    format_str = "%(asctime)s - %(levelname)s - %(message)s"
    # configure
    if fn_out is not None:
        logging.basicConfig(
            filename = fn_out,
            filemode = "w",
            format = format_str,
            level = logging.DEBUG
        )
    else:
        logging.basicConfig(
            format = format_str,
            level = logging.DEBUG
        )
        
    logger = logging.getLogger(namespace)
    # create console handler and set level to debug
    ch = logging.StreamHandler()
    ch.setLevel(logging.DEBUG)
    # create formatter
    formatter = logging.Formatter(format_str)
    # add formatter to ch
    ch.setFormatter(formatter)
    # add ch to logger
    logger.addHandler(ch)

    return logger

_setup_logger(__name__, os.path.join(os.getcwd(), "log_turkey_data_buil.log"))

/Users/jsyme/Documents/Projects/git_jbus/sisepuede/python/model_attributes.py:2635: UserWarning: Invalid subsector attribute 'key_varreqs_partial'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector', 'key_varreqs_all'
  warnings.warn(f"Invalid subsector attribute '{return_type}'. Valid return type values are:{valid_rts}")
/Users/jsyme/Documents/Projects/git_jbus/sisepuede/python/model_attributes.py:2392: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for desig, df in df_by_designation:
/Users/jsyme/Documents/Projects/git_jbus/sisepuede/python/model_attributes.py:2635: UserWarning: Invalid subsector attribute 'key_varreqs_partial'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector', 'key_varreqs_all'
  warnings.warn(f"Invalid subsector a

<Logger __main__ (DEBUG)>

In [133]:
# some data
dir_data_turkey = "/Users/jsyme/Documents/Personal Files/Other Work/WB_SISEPUEDE/Turkey/turkey_lts_model_data"
fp_power = os.path.join(dir_data_turkey, "POWER sector_Phase 2_MENRto2035.xlsx")


field_zone = "zone"
field_fuel = "fuel"
regex_exists = re.compile("(.*)Ex($)")

time_periods = sc.TimePeriods(sa.model_attributes)


df_capacity = (
    pd.read_excel(
        fp_power, 
        sheet_name = "CapacityByFuel", 
        skiprows = 4
    )
    .rename(
        columns = {
            "Unnamed: 0": field_zone,
            "Unnamed: 1": field_fuel,
        }
    )
)



def clean_fuel_name(
    fuel_name: str,
    regex_exists: re.Pattern,
    str_split: Union[str, None],
    type_exists: str = "existing",
    type_new: str = "new",
) -> Tuple[str]:

    if not isinstance(fuel_name, str):
        return ("", "")

    fuel_name = (
        (fuel_name.split(str_split)[1] if (str_split in fuel_name) else fuel_name)
        if isinstance(str_split, str)
        else fuel_name
    )

    # check match
    match = regex_exists.match(fuel_name)
    type_capacity = type_new
    if match is not None:
        type_capacity = type_exists
        fuel_name = match.groups()[0]

    tup_out = (fuel_name, type_capacity)

    return tup_out
    
    
    
def clean_fuel(
    series_fuel: pd.Series,
    field_fuel: str = field_fuel,
    field_type: str = "capacity_type",
    regex_exists: re.Pattern = regex_exists,
    str_split: Union[str, None] = ".",
    type_exists: str = "existing",
    type_new: str = "new",
) -> pd.DataFrame:
    """
    Clean fuel names from turkey LTS data. Matches capacity for existing
        fuels based on regex_exists
    """
    
    capac = list(series_fuel)
    
    """
    clean_fuel_name(
        fuel_name: str,
        regex_exists,
        str_split: Union[str, None],
        type_exists: str = "existing",
        type_new: str = "new",
    )
    """;
    
    out = series_fuel.apply(
        clean_fuel_name, 
        args = (regex_exists, str_split),
        type_exists = type_exists,
        type_new = type_new,
    )
    
    out = pd.DataFrame(
        list(out), 
        columns = [field_fuel, field_type]
    )
    
    return out



def prepare_data_frame(
    df_in: pd.DataFrame,
    dict_replace_fuels: Union[Dict[str, str], None] = None,
    field_fuel: str = field_fuel,
    field_type: str = "capacity_type",
    **kwargs
) -> pd.DataFrame:
    """
    Clean the capacity data frame
    
    Keyword Arguments
    -----------------
    - dict_replace_fuels: optional dictionary to replace fuels with new name.
    - **kwargs: passed to clean_fuel
    """
    
    ##  SETUP FUEL
    
    df_fuel = clean_fuel(
        df_in[field_fuel],
        field_fuel = field_fuel,
        field_type = "capacity_type",
        **kwargs
    )
    
    (
        df_fuel[field_fuel].replace(dict_replace_fuels, inplace = True)
        if isinstance(dict_replace_fuels, dict)
        else None
    )
    
    
    ##  AGGREGATE
    
    df_out = pd.concat(
        [
            df_in.drop([field_fuel], axis = 1),
            df_fuel
        ],
        axis = 1
    )
    
    fields_dat = [
        x for x in df_out.columns 
        if x.isnumeric()
    ]
    df_out = (
        sf.simple_df_agg(
            df_out,
            [field_fuel, field_type],
            dict((x, "sum") for x in fields_dat)
        )
    )
    
    
    ##  MELT, INTERPOLATE, AND PIVOT
    
    field_value = "value"
    df_out = pd.melt(
        df_out,
        [field_fuel, field_type],
        fields_dat,
        var_name = time_periods.field_year,
        value_name = field_value,
    )
    
    df_out[time_periods.field_year] = df_out[time_periods.field_year].astype(int)
    
    # build combination of all years/powerplants
    df_years = (
        time_periods
        .get_time_period_df(include_year = True)
        [[time_periods.field_year]]
    )
    df_years = sf.explode_merge(
        df_years,
        df_out[[field_fuel, field_type]].drop_duplicates()
    )
    
    
    # merge into all years
    df_out = pd.merge(
        df_years,
        df_out,
        how = "left"
    )
    
    # group by tech and interpolate
    df_out_group = df_out.groupby([field_fuel, field_type])
    df_out_new = []
    for tech, df in df_out_group:
        
        # sort and interpolate
        df.sort_values(
            by = [time_periods.field_year],
            inplace = True,
        )
        df[field_value] = (
            df[field_value]
            .interpolate()
            .interpolate(method = "bfill")
        )
        
        df_out_new.append(df)
        
    
    # re-concatenate
    df_out = (
        pd.concat(
            df_out_new,
            axis = 0
        )
        .sort_values(
            by = [field_fuel, field_type, time_periods.field_year]
        )
        .reset_index(
            drop = True
        )
    )
   
    # pivot
    df_out = (
        sf.pivot_df_clean(
            df_out,
            [time_periods.field_year],
            [field_value],
        )
        .sort_values(
            by = [field_type, field_fuel]
        )
        .reset_index(drop = True)
    )
    
    
    return df_out


# replace techs
dict_pp_tech = {
    "Asphaltite": "pp_oil",
    "Battery": "st_batteries",
    "Biomass": "pp_biomass",
    "CSP": "pp_solar",
    "CoalCCS": "pp_coal_ccs",
    "FuelOil": "pp_oil",
    "Gas": "pp_gas",
    "GasCCS": "pp_gas_ccs",
    "Geo": "pp_geothermal",
    "Hydrogen": "pp_solar", # TEMPORARY- NO HYDROGEN
    "ImportedCoal": "pp_coal",
    "Lignite": "pp_coal",
    "NationalCoal": "pp_coal",
    "Offshore": "pp_wind",
    "PHP": "pp_solar", # DON'T KNOW WHAT THIS IS
    "PV": "pp_solar",
    "Uranium": "pp_nuclear",
    "Water": "pp_hydro",
    "Wind": "pp_wind",
}



df_temp = prepare_data_frame(df_capacity, dict_replace_fuels = dict_pp_tech)
df_temp.to_csv(
    os.path.join(dir_data_turkey, "phase_ii_capacities.csv"),
    index = None, 
    encoding = "UTF-8",
)
    

In [140]:
df_capacity[
    df_capacity["fuel"].isin(["10.CoalCCS"])
    
]

,zone,fuel,2022,2025,2030,2035,2040,2045,2050,2053
9,Anatolia,10.CoalCCS,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001
38,Trakya,10.CoalCCS,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001


,strategy_id,strategy,strategy_code,baseline_strategy_id,description
0,0,Baseline NDP,BASE,1,Base assumptions to which all transformations ...
1,1001,AGRC: All transformations,AGRC:ALL,0,All agricultural transformations
2,1002,AGRC: All transformations with partial land us...,AGRC:ALL_PLUR,0,All agricultural transformations
3,1003,AGRC: Decrease Exports,AGRC:DEC_EXPORTS,0,Decrease agricultural exports by 50% (relative...
4,1004,AGRC: Expand conservation agriculture,AGRC:INC_CONSERVATION_AGRICULTURE,0,| Decrease soil organic carbon loss in cropla...
...,...,...,...,...,...
123,5005,PFLO: Industrial point of capture CCSQ,PFLO:IND_INC_CCS,0,Specifies point-of-capture carbon capture and ...
124,5006,PFLO: Supply side technologies and transformat...,PFLO:SUPPLY_SIDE_TECH,0,Macro-scale transformations without consumer b...
125,5007,PFLO: All transformations without stopping def...,PFLO:ALL_NO_STOPPING_DEFORESTATION_PLUR,0,All transformations and partial land use reall...
126,5008,PFLO: All transformations,PFLO:ALL,0,All transformations


In [147]:
n = 15
delta = 0.75/n
for i in range(1, n+1):
    print(np.round(delta*i, decimals = 4))

0.05
0.1
0.15
0.2
0.25
0.3
0.35
0.4
0.45
0.5
0.55
0.6
0.65
0.7
0.75
